# Recognising NRs


In [7]:
import sys
import os
basef = "/mnt/data/Anastasia/"
wdirs = ["/mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/",
         basef+"18_12_10_pd3_12_div7_WIS_NR-BeRST",
         basef+"18_12_12_pd7_12_div5_WIS_NR-BeRST",
         basef+"18_12_12_pd7_12_div5_WIS_NR-BeRST_DM590",
         basef+"19_01_30_pd25_01_div5_NR_BeRST",
         basef+"19_02_05_pd1_02_div4_NR_BeRST"
]

dfiles = []
for dirt in wdirs:
    basedir = dirt
    files = os.listdir(basedir)
    if dirt[-1] != '/':
        dirt = dirt+'/'
    for f in files:
        if f[-4:]=='.tif': 
            try:
                i = int(f[-5])
                dfiles.append(dirt+f)
            except:
                pass

            
     
               
  
            
#dfiles = dfiles[4:8]

print(len(dfiles))
print(dfiles)


35
['/mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell2_3.tif', '/mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell1_1.tif', '/mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell3_5.tif', '/mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell4_7.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell5_10.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell2_3.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell1_1.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell4_8.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell7_14.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell6_12.tif', '/mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell3_5.tif', '/mnt/data/Anastasia/18_12_12_pd7_12_div5_WIS_NR-BeRST/cell1_1.tif', '/mnt/data/Anastasia/18_12_12_pd7_12_div5_WIS_NR-BeRST/cell3_7.tif', '/mnt/data/Anastasia/18_12_12_pd7_12_div5_WIS_NR-BeRST/cell2_4.tif', '/mnt/data/Anastasia/18_12_12_p

In [8]:
sys.path.append("/export/home1/users/bssn/serna/GitIBENS/Nanorods")

In [9]:
from numpy import *
from matplotlib.pylab import *
import scipy.optimize as opt
import pickle
from scipy.optimize import minimize,least_squares
from scipy.stats import linregress

%matplotlib inline

In [10]:
from signal_analysis.core import dblgausfit,dblgaussd

In [21]:
for i,cfile in enumerate(dfiles):
    wdir = ''
    cf2 = cfile.split(".")[0].split("/")
    for fs in cf2[:-1]:
        wdir = wdir+fs+'/'
    mfile = wdir+cf2[-1]+'output/ROIs_Inmask.dat'
    mt = loadtxt(mfile)
    print(mt.shape,mfile)
    if i == 0:
        imsk = mt.flatten()
    else:
        imsk = concatenate((imsk,mt.flatten()))

(1333,) /mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell2_3output/ROIs_Inmask.dat
(1623,) /mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell1_1output/ROIs_Inmask.dat
(860,) /mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell3_5output/ROIs_Inmask.dat
(1495,) /mnt/data/Anastasia/18_11_29_pd23_11_div6_25Hzsqwave/cell4_7output/ROIs_Inmask.dat
(376,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell5_10output/ROIs_Inmask.dat
(4426,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell2_3output/ROIs_Inmask.dat
(5765,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell1_1output/ROIs_Inmask.dat
(224,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell4_8output/ROIs_Inmask.dat
(3874,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell7_14output/ROIs_Inmask.dat
(7559,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell6_12output/ROIs_Inmask.dat
(8985,) /mnt/data/Anastasia/18_12_10_pd3_12_div7_WIS_NR-BeRST/cell3_5output/ROIs_Inm

In [12]:
def bimod(x):
    m1 = mean(x)
    m2 = std(x)
    m3 = mean((x-m1)**3)/m2**3
    m4 = mean((x-m1)**4)/m2**4
    return((m3**2+1)/m4)

In [13]:
from msd import extractmsd,extractDe



In [12]:
dataL = []
nwdirs = []
for i,cfile in enumerate(dfiles):
    wdir = ''
    cf2 = cfile.split(".")[0].split("/")
    for fs in cf2[:-1]:
        wdir = wdir+fs+'/'
    wdir = wdir+cf2[-1]+'output/sptrack/'
    nwdirs.append(wdir)
    
    with open(wdir+'data_msd.pickle', 'rb') as handle:
        datat = pickle.load(handle)
    dataL = [datat]


    xdst,xdet,xdent = extractmsd(dataL,verbose=True)


    Dest, Dept = extractDe(dataL,xdet,xdst)
    print(i)
    xdst[:,-1] = i
    xdet[:,-1] = i
    xdent[:,-1] = i
    Dest[:,-1] = i
    Dept[:,-1] = i
    if i == 0:
        xds = xdst
        xde = xdet
        xden = xdent
        Des = Dest
        Dep = Dept
    else:
        xds = row_stack((xds,xdst))
        xde = row_stack((xde,xdet))
        xden = row_stack((xden,xdent))
        Des = row_stack((Des,Dest))
        Dep = row_stack((Dep,Dept))
        
    #dataL.append(datat)

/export/home1/users/bssn/serna/GitIBENS/Nanorods/sptrack/Classifying NRs/msd.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  seb = sqrt((ssx*ssy-ssxy**2)/((n-2)))/ssx
/export/home1/users/bssn/serna/GitIBENS/Nanorods/sptrack/Classifying NRs/msd.py:34: RuntimeWarning: invalid value encountered in double_scalars
  b = ssxy/ssx
/export/home1/users/bssn/serna/GitIBENS/Nanorods/sptrack/Classifying NRs/msd.py:35: RuntimeWarning: invalid value encountered in double_scalars
  seb = sqrt((ssx*ssy-ssxy**2)/((n-2)))/ssx
/export/home1/users/bssn/serna/GitIBENS/Nanorods/sptrack/Classifying NRs/msd.py:35: RuntimeWarning: invalid value encountered in sqrt
  seb = sqrt((ssx*ssy-ssxy**2)/((n-2)))/ssx


0


/export/home1/users/bssn/serna/GitIBENS/Nanorods/sptrack/Classifying NRs/msd.py:139: RuntimeWarning: invalid value encountered in true_divide
  msds = msds[1:,:]/msds[:-1,:]
/users/bssn/serna/environments/deepl/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/bssn/serna/environments/deepl/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0
0
0
0
1
0
0
2
0
0
3
0
0
4
0
0
5
0
0
6
0
0
7
0
0
8
0
0
9
0
0
10
0
0
11
0
0
12
0
0
13
0
0
14
0
0
15


In [13]:
with open('/mnt/data/Anastasia/DesDep19on.pkl','wb') as f:
    pickle.dump([Des,Dep],f)

In [22]:
with open('/mnt/data/Anastasia/full_stats.pkl','rb') as f:
    #pickle.dump([idx0,dgps,ampst,xds,xde,xde,nDes,Dep],f)
    _,nwdirs,dgps,ampst,idx0,xds,xde,xden,Des,Dep = pickle.load(f)

In [23]:
# Lo estas haciendo bien? Comprueba!!

#with open('/mnt/data/Anastasia/full_stats.pkl','wb') as f:
#    #pickle.dump([idx0,dgps,ampst,xds,xde,xde,nDes,Dep],f)
#    pickle.dump([imsk,nwdirs,dgps,ampst,idx0,xds,xde,xden,Des,Dep],f)

In [16]:
imsk.sum()

5011.0

In [20]:
imsk[:672]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0.